In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import sys
sys.path.append("..")
import os
import shutil
from utils.model3 import Net_mask
#from utils.data import Data
#from scripts.parameters import get_parameters, get_parameters_1
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.distributions.multivariate_normal import MultivariateNormal

In [12]:
n_data = 1
n_param = 10
sigma2_data = 1
data_mean = torch.zeros(n_param)
data_covariance = torch.eye(n_param)*sigma2_data

torch.manual_seed(0)
n_data_points = 6
n_points_max = 6

all_points = torch.zeros(100, 20)
all_solutions = torch.zeros(100, 10)

for i in range(n_points_max):
    for j in range(n_data_points):
        torch.manual_seed(f'{i+1}{j}')
        tensor_data = torch.zeros(n_data, 2*n_param)
        tensor_d_sample =  MultivariateNormal(loc=data_mean, covariance_matrix=data_covariance).sample(sample_shape=(n_data,))
        #tensor_n_masked = torch.randint(n_param, (n_data,))
        tensor_n_masked = torch.Tensor([i+1])
        #print(tensor_n_masked)
        tensor_masks = torch.rand(n_data, n_param).argsort(dim=1)
        tensor_masks = (tensor_masks < tensor_n_masked.unsqueeze(1))*1
        tensor_data[:,:n_param] = tensor_d_sample*tensor_masks
        tensor_data[:,n_param:] = tensor_masks
        all_points[int(f'{i+1}{j}'),:] = tensor_data
        #if i==4 and j==3:
        #    print(tensor_data)
        #torch.save(tensor_data, f'../data/test_observations/{i+1}_{j}.pt')
torch.save(all_points, f'../data/test_observations/all_test_observations.pt')

In [13]:
tensor_D = torch.tensor(
    [[1, -1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, -1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0,  1, -1, 0, 0, 0, 0, 0, 0],
    [0, 0,  0, 1, -1, 0, 0, 0, 0, 0],
    [0, 0,  0, 0, 1, -1, 0, 0, 0, 0],
    [0, 0,  0, 0, 0, 1, -1, 0, 0, 0],
    [0, 0,  0, 0, 0, 0, 1, -1, 0, 0],
    [0, 0,  0, 0, 0, 0, 0, 1, -1, 0],
    [0, 0,  0, 0, 0, 0, 0, 0, 1, -1],
    [-1, 0, 0, 0, 0, 0, 0, 0, 0, 1],], dtype=torch.float
)
tensor_Q_m = torch.mm(torch.t(tensor_D), tensor_D) 
n_param = tensor_D.size(dim=0)
vu = 10000
inversion_scaling = 1./vu
tensor_Q_m_modified = tensor_Q_m + torch.ones(n_param, n_param)*inversion_scaling


tensor_eval = torch.zeros(n_data, 2*n_param)

sigma2 = 0.01
tensor_mu_m = torch.zeros(n_param)
tensor_Sigma_m = torch.inverse(tensor_Q_m_modified)
tensor_Sigma_eps = torch.eye(n_param)*sigma2
tensor_mu_eps = tensor_mu_m 

all_solutions = torch.zeros(100, 10)

for i in range(n_points_max):
    for j in range(n_data_points):
        input_tensor = all_points[int(f'{i+1}{j}'),:]
        d = input_tensor[:n_param]
        mask = input_tensor[n_param:]
        location = torch.where(mask > 0.5)
        d = d[location]
        n_data = len(d)
        G = torch.zeros(n_data, n_param)
        for idx, elem in enumerate(location[0]):
            G[idx, elem] = 1
        Sigma_eps = torch.matmul(G, torch.matmul(tensor_Sigma_eps, torch.t(G)))
        mu_eps = torch.matmul(G, tensor_mu_eps)

        #matrix calculations
        Sigma_mm = tensor_Sigma_m
        Sigma_dm = torch.matmul(G,Sigma_mm)
        Sigma_md = torch.matmul(Sigma_mm, torch.t(G))
        Sigma_dd = torch.matmul(G, torch.matmul(Sigma_mm, torch.t(G))) + Sigma_eps
        Sigma_dd_inv = torch.inverse(Sigma_dd)
        mu_m_d = tensor_mu_m + torch.matmul(torch.matmul(Sigma_md, Sigma_dd_inv),(d-mu_eps))
        Sigma_m_d = Sigma_mm - torch.matmul(torch.matmul(Sigma_md, Sigma_dd_inv), Sigma_dm)
        all_solutions[int(f'{i+1}{j}'),:] = mu_m_d
torch.save(all_solutions, f'../data/test_observations/all_test_solutions.pt')